In [1]:
push!(LOAD_PATH,".")

5-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "/home/skh/MultiAgentSensing"
 "."

In [2]:
using Revise
using POMDPs
using POMDPSimulators

In [3]:
using MDMA

┌ Info: Precompiling MDMA [top-level]
└ @ Base loading.jl:1423


Test Summary: | Pass  Total
detectTarget  |   30     30
Test Summary:        | Pass  Total
single_robot_planner |    2      2


In [124]:
sensor = ViewConeSensor(pi/2, 3);
targets = Vector{Target}(undef, 0);
push!(targets, Target(10,10,0));
push!(targets, Target(10,11,0));
push!(targets, Target(11,11,0));

grid = Grid(11,11);
horizon = 30
move_dist = 3
model = MDMA.SingleRobotMultiTargetViewCoverageProblem(grid, sensor, horizon, targets, move_dist);

In [125]:
policy = MDMA.solve_single_robot(model, UAVState(1,1,:N))


[Iteration 1   ] residual:        100 | iteration runtime:    341.274 ms, (     0.341 s total)
[Iteration 2   ] residual:        100 | iteration runtime:    316.953 ms, (     0.658 s total)
[Iteration 3   ] residual:         90 | iteration runtime:    319.477 ms, (     0.978 s total)
[Iteration 4   ] residual:         90 | iteration runtime:    318.252 ms, (       1.3 s total)
[Iteration 5   ] residual:         90 | iteration runtime:    313.191 ms, (      1.61 s total)
UAVState(3, 3, :NE, false)nothing

ValueIterationPolicy:
 MDPState(UAVState(1, 1, :E, false), 0, nothing) -> UAVState(3, 1, :SE, false)
 MDPState(UAVState(1, 2, :E, false), 0, nothing) -> UAVState(3, 1, :SE, false)
 MDPState(UAVState(1, 3, :E, false), 0, nothing) -> UAVState(3, 1, :SE, false)
 MDPState(UAVState(1, 4, :E, false), 0, nothing) -> UAVState(3, 2, :SE, false)
 MDPState(UAVState(1, 5, :E, false), 0, nothing) -> UAVState(3, 3, :SE, false)
 MDPState(UAVState(1, 6, :E, false), 0, nothing) -> UAVState(3, 4, :SE, false)
 MDPState(UAVState(1, 7, :E, false), 0, nothing) -> UAVState(3, 5, :SE, false)
 MDPState(UAVState(1, 8, :E, false), 0, nothing) -> UAVState(3, 6, :SE, false)
 MDPState(UAVState(1, 9, :E, false), 0, nothing) -> UAVState(3, 7, :SE, false)
 MDPState(UAVState(1, 10, :E, false), 0, nothing) -> UAVState(3, 8, :SE, false)
 MDPState(UAVState(1, 11, :E, false), 0, nothing) -> UAVState(3, 9, :SE, false)
 MDPState(UAVState(2, 1, :E, false), 0, nothing) -> UAVState(3, 1, :SE, false)
 MDPState(UAVState(2, 2, :E,

In [126]:
function rew_check(model, state, policy)
    prev = MDPState(state)
    next = action(policy, prev)
    rew1 = MDMA.reward(model, prev, prev.state)
    rew2 = MDMA.reward(model, prev, next)
    println(prev.state," ", rew1)
    println(next," ", rew2)
end
function check(model, prev, next)
    prev = MDPState(prev)
    rew1 = MDMA.reward(model, prev, prev.state)
    rew2 = MDMA.reward(model, prev, next)
    println(prev.state," ", rew1)
    println(next," ", rew2)
end


check (generic function with 1 method)

In [127]:
test_state = UAVState(3,3,:S)
rew_check(model, test_state, policy)

UAVState(3, 3, :S, false) 0
UAVState(3, 3, :SW, false) 0


In [128]:
check(model, test_state, UAVState(3,3,:E))

UAVState(3, 3, :S, false) 0
UAVState(3, 3, :E, false) 0


In [129]:
nbors = MDMA.neighbors(model.grid, test_state, move_dist)

75-element Vector{UAVState}:
 UAVState(1, 1, :SE, false)
 UAVState(1, 1, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(1, 2, :SE, false)
 UAVState(1, 2, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(1, 3, :SE, false)
 UAVState(1, 3, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(1, 4, :SE, false)
 UAVState(1, 4, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(1, 5, :SE, false)
 ⋮
 UAVState(5, 2, :SE, false)
 UAVState(5, 2, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(5, 3, :SE, false)
 UAVState(5, 3, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(5, 4, :SE, false)
 UAVState(5, 4, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(5, 5, :SE, false)
 UAVState(5, 5, :SW, false)
 UAVState(3, 3, :S, false)

In [130]:
UAVState(3,3,:E) in nbors

false

In [131]:
test_state in nbors

true

In [132]:
nbors

75-element Vector{UAVState}:
 UAVState(1, 1, :SE, false)
 UAVState(1, 1, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(1, 2, :SE, false)
 UAVState(1, 2, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(1, 3, :SE, false)
 UAVState(1, 3, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(1, 4, :SE, false)
 UAVState(1, 4, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(1, 5, :SE, false)
 ⋮
 UAVState(5, 2, :SE, false)
 UAVState(5, 2, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(5, 3, :SE, false)
 UAVState(5, 3, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(5, 4, :SE, false)
 UAVState(5, 4, :SW, false)
 UAVState(3, 3, :S, false)
 UAVState(5, 5, :SE, false)
 UAVState(5, 5, :SW, false)
 UAVState(3, 3, :S, false)

In [133]:
path = Vector{MDPState}(undef, 0)
state = MDPState(UAVState(1,1,:S))
push!(path, state)
for x in 1:100
    @show state
    state = action(policy, state)
    state = MDPState(state)
    push!(path, state)
    println(reward(model, state, state.state))
end
path

state = MDPState(UAVState(1, 1, :S, false), 0, nothing)
0
state = MDPState(UAVState(3, 3, :SW, false), 0, nothing)
0
state = MDPState(UAVState(5, 5, :W, false), 0, nothing)
0
state = MDPState(UAVState(7, 7, :NW, false), 0, nothing)
15
state = MDPState(UAVState(9, 9, :N, false), 0, nothing)
15
state = MDPState(UAVState(9, 9, :NE, false), 0, nothing)
15
state = MDPState(UAVState(8, 11, :E, false), 0, nothing)
5
state = MDPState(UAVState(8, 11, :SE, false), 0, nothing)
5
state = MDPState(UAVState(9, 11, :S, false), 0, nothing)
10
state = MDPState(UAVState(11, 11, :SW, false), 0, nothing)
10
state = MDPState(UAVState(11, 10, :W, false), 0, nothing)
15
state = MDPState(UAVState(11, 9, :NW, false), 0, nothing)
15
state = MDPState(UAVState(9, 9, :N, false), 0, nothing)
15
state = MDPState(UAVState(9, 9, :NE, false), 0, nothing)
15
state = MDPState(UAVState(8, 11, :E, false), 0, nothing)
5
state = MDPState(UAVState(8, 11, :SE, false), 0, nothing)
5
state = MDPState(UAVState(9, 11, :S, false), 

101-element Vector{MDPState}:
 MDPState(UAVState(1, 1, :S, false), 0, nothing)
 MDPState(UAVState(3, 3, :SW, false), 0, nothing)
 MDPState(UAVState(5, 5, :W, false), 0, nothing)
 MDPState(UAVState(7, 7, :NW, false), 0, nothing)
 MDPState(UAVState(9, 9, :N, false), 0, nothing)
 MDPState(UAVState(9, 9, :NE, false), 0, nothing)
 MDPState(UAVState(8, 11, :E, false), 0, nothing)
 MDPState(UAVState(8, 11, :SE, false), 0, nothing)
 MDPState(UAVState(9, 11, :S, false), 0, nothing)
 MDPState(UAVState(11, 11, :SW, false), 0, nothing)
 MDPState(UAVState(11, 10, :W, false), 0, nothing)
 MDPState(UAVState(11, 9, :NW, false), 0, nothing)
 MDPState(UAVState(9, 9, :N, false), 0, nothing)
 ⋮
 MDPState(UAVState(11, 11, :SW, false), 0, nothing)
 MDPState(UAVState(11, 10, :W, false), 0, nothing)
 MDPState(UAVState(11, 9, :NW, false), 0, nothing)
 MDPState(UAVState(9, 9, :N, false), 0, nothing)
 MDPState(UAVState(9, 9, :NE, false), 0, nothing)
 MDPState(UAVState(8, 11, :E, false), 0, nothing)
 MDPState(UAV

In [134]:
using Cairo



In [148]:
using Cairo

#Pixels per meter
ppm = 50
width = ppm*dims(model.grid)[1]+ppm
height = ppm*dims(model.grid)[2]+ppm
c = CairoRGBSurface(width,height);
cr = CairoContext(c);

function draw_background(cr::CairoContext, width, height)
    save(cr);
    set_source_rgb(cr,1,1,1);    # white
    rectangle(cr,0.0,0.0,width,height); # background
    fill(cr);
    restore(cr);
end

function draw_grid(g::Grid, cr::CairoContext, point_size, ppm)
    set_source_rgba(cr, 0, 0, 0, 0.3);
    x,y,z = dims(g)
    print(dims(g))
    for i in 1:x
        for j in 1:y
            arc(cr, i*ppm, j*ppm, point_size, 0, 2*pi);
            fill(cr);
        end
    end    
end

function draw_target(cr::CairoContext, t::Target)
    set_source_rgba(cr, 0, 0.3, 0.5, 1);
    arc(cr, t.x*ppm, t.y*ppm, 10, 0, 2*pi);
    fill(cr);
end

function draw_targets(cr::CairoContext, targs::Vector{Target})
    for t in targs
        draw_target(cr, t)
    end
end

function draw_state(cr::CairoContext, state::UAVState, model, ppm, fade)
    fov = model.sensor.fov
    radius = model.sensor.cutoff
    draw_arc(cr, radius, state.x, state.y, state.heading, fov, ppm, fade)
end
function draw_arc(cr::CairoContext, radius, x,y, heading,fov,ppm, fade) 
    ## original example, following here
    
    xc = x*ppm;
    yc = y*ppm;
    radius = radius*ppm;
    angle1 = dirAngle(heading) + (-fov/2);  # angles are specified
    angle2 = dirAngle(heading) + (fov/2);  # in radians

    set_line_width(cr, 10.0);
    arc(cr, xc, yc, radius, angle1, angle2);
    stroke(cr);

    # draw helping lines
    set_source_rgba(cr, 1, 0.2, 0.2, fade);
    set_line_width(cr, 6.0);

    # Draw center
    arc(cr, xc, yc, 10.0, 0, 2*pi);
    fill(cr);
    
    set_source_rgba(cr, 0, 0, 0, fade);
    arc(cr, xc, yc, radius, angle1, angle1);
    set_source_rgba(cr, 1, 0.2, 0.2, fade);
    line_to(cr, xc, yc);
    set_source_rgba(cr, 0, 0, 0, fade);
    arc(cr, xc, yc, radius, angle2, angle2);
    set_source_rgba(cr, 1, 0.2, 0.2, fade);
    line_to(cr, xc, yc);
    stroke(cr);
end
draw_background(cr, width, height)
draw_grid(model.grid, cr, 5, ppm)
# draw_state(cr, UAVState(1,1,:S), model, ppm)


## mark picture with current date


(11, 11, 8)

In [161]:
state= path[1]
move_to(cr,state.state.x*ppm, state.state.y*ppm)
draw_state(cr, state.state, model, ppm, 0.8)
set_line_width(cr, 10.0);
stroke(cr);
for (i,state) in enumerate(path[2:10])
    println(i)
    line_to(cr, state.state.x*ppm, state.state.y*ppm)
    draw_state(cr, state.state, model, ppm, 0.8)
end

1
2
3
4
5
6
7
8
9


In [162]:
draw_targets(cr, model.targets)
write_to_png(c,"sample_arc.png");